In [7]:
import pandas as pd
import os
from caveclient import CAVEclient
#import cloudvolume
import numpy as np
#import requests
import matplotlib.pyplot as plt
%matplotlib inline
from nglui.statebuilder import *
import utils_EL as ut
client = CAVEclient()
if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
    client.auth.get_new_token(open=True)
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [10]:
premotor_df = pd.read_csv('/Users/ellenlesser/Desktop/pmn_env/CSVs/premotor0816.csv')
sensory_df = premotor_df[premotor_df['class'].isin(['sensory'])]
descending_df = premotor_df[premotor_df['class'].isin(['descending'])]
local_df = premotor_df[premotor_df['class'].isin(['local'])]

In [17]:
# Designate which neuron(s) to query
# output if segIDs have no partners - means the ID is out of date
# otherwise output top 20 partners & their # of synapses with the queried neruon(s)

thresh_syn = 3 # minimum synapse threshold to be considered a partner
prepost = "pre" # "pre" to find presynaptic partners, "post" to find postsynaptic partners
query_list = [648518346487932240]#[648518346477534222,648518346487932240]

synapse_df = ut.build_synapse_df(query_list,prepost,thresh_syn,client) 
postcounts = pd.DataFrame(synapse_df['post_pt_root_id'].value_counts().rename_axis('SegID').reset_index(name='counts'))
precounts = pd.DataFrame(synapse_df['pre_pt_root_id'].value_counts().rename_axis('SegID').reset_index(name='counts'))

if postcounts.shape[0] != len(query_list):
    print('Warning: The following segIDs are not up to date')
    print(list(set(query_list) - set(postcounts['SegID'])))

In [12]:
synapse_df.pre_pt_root_id.value_counts()

648518346514394055    82
648518346483584149    70
648518346472167218    56
648518346515433210    55
648518346483521455    54
                      ..
648518346486727504     3
648518346387679228     3
648518346486861392     3
648518346501146712     3
648518346514100779     3
Name: pre_pt_root_id, Length: 361, dtype: int64

In [6]:
# create a neuroglancer state with different colored synapses for top x presynaptic partners
## LOCAL
df = synapse_df
df = df[df.pre_pt_root_id.isin(local_df.SegID.to_list())]
precounts = pd.DataFrame(df['pre_pt_root_id'].value_counts().rename_axis('SegID').reset_index(name='counts'))

#reds = ['#fdd4c2', '#fca082', '#fb694a', '#e32f27', '#b11218']
reds = ['#b11218', '#e32f27', '#fb694a', '#fca082', '#fdd4c2'] #dark to light
blues = ['#00035b', '#0504aa', '#0165fc', '#0485d1', '#a2cffe' ] #dark to light
greens = ['#06470c', '#658b38', '#15b01a', '#96f97b', '#c7fdb5']
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']
# df['color'] = reds

img_source = client.info.image_source()
img_layer = ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()
seg_layer = SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = query_list)#precounts.SegID[0:2])
points = PointMapper(point_column='pre_pt_position', set_position =True) ######################## change this to toggle rendering of pre- or post- synaptic points

## 1 ##
i = 0
syn_1_L = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer1L = AnnotationLayerConfig(name='annos1L', color=purples[i],
                                   mapping_rules=points)
sb1_L = StateBuilder(layers = [img_layer, seg_layer, anno_layer1L], resolution=[4.3,4.3,45])
i = i +1

## 2 ##
syn_2_L = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer2L = AnnotationLayerConfig(name='annos2L', color=purples[i],
                                   mapping_rules=points)
sb2_L = StateBuilder(layers = [anno_layer2L], resolution=[4.3,4.3,45])
i = i +1

## 3 ##
syn_3_L = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer3L = AnnotationLayerConfig(name='annos3L', color=purples[i],
                                   mapping_rules=points)
sb3_L = StateBuilder(layers = [anno_layer3L], resolution=[4.3,4.3,45])
i = i+1

## 4 ##
syn_4_L = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer4L = AnnotationLayerConfig(name='annos4L', color=purples[i],
                                   mapping_rules=points)
sb4_L = StateBuilder(layers = [anno_layer4L], resolution=[4.3,4.3,45])
i = i+1

## 5 ##
syn_5_L = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer5L = AnnotationLayerConfig(name='annos5L', color=purples[i],
                                   mapping_rules=points)
sb5_L = StateBuilder(layers = [anno_layer5L], resolution=[4.3,4.3,45])
i = i+1

local_states = [sb1_L,sb2_L,sb3_L,sb4_L,sb5_L]
local_syns = [syn_1_L,syn_2_L,syn_3_L,syn_4_L,syn_5_L]
# chained_sb_L = ChainedStateBuilder([sb1,sb2,sb3,sb4,sb5])
# chained_sb.render_state([syn_1,syn_2,syn_3,syn_4,syn_5], return_as='html')

In [160]:
# create a neuroglancer state with different colored synapses for top x presynaptic partners
## DESCENDING
df = synapse_df
df = df[df.pre_pt_root_id.isin(descending_df.SegID.to_list())]
precounts = pd.DataFrame(df['pre_pt_root_id'].value_counts().rename_axis('SegID').reset_index(name='counts'))

# #reds = ['#fdd4c2', '#fca082', '#fb694a', '#e32f27', '#b11218']
# reds = ['#b11218', '#e32f27', '#fb694a', '#fca082', '#fdd4c2'] #dark to light
# blues = ['#00035b', '#0504aa', '#0165fc', '#0485d1', '#a2cffe' ] #dark to light
# greens = ['#06470c', '#658b38', '#15b01a', '#96f97b', '#c7fdb5']
# purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']
# # df['color'] = reds

# img_source = client.info.image_source()
# img_layer = ImageLayerConfig(name='fanc_v4',
#                              source=img_source,
#                              )
# seg_source = client.info.segmentation_source()
# seg_layer = SegmentationLayerConfig(name = 'seg',
#                                     source = seg_source,
#                                     fixed_ids = query_list)#precounts.SegID[0:2])
# points = PointMapper(point_column='pre_pt_position', set_position =True) ######################## change this to toggle rendering of pre- or post- synaptic points

## 1 ##
i = 0
syn_1_D = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer1D = AnnotationLayerConfig(name='annos1D', color=blues[i],
                                   mapping_rules=points)
# sb1_D = StateBuilder(layers = [img_layer, seg_layer, anno_layer1D], resolution=[4.3,4.3,45])
sb1_D = StateBuilder(layers = [anno_layer1D], resolution=[4.3,4.3,45])

i = i +1

## 2 ##
syn_2_D = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer2D = AnnotationLayerConfig(name='annos2D', color=blues[i],
                                   mapping_rules=points)
sb2_D = StateBuilder(layers = [anno_layer2D], resolution=[4.3,4.3,45])
i = i +1

## 3 ##
syn_3_D = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer3D = AnnotationLayerConfig(name='annos3D', color=blues[i],
                                   mapping_rules=points)
sb3_D = StateBuilder(layers = [anno_layer3D], resolution=[4.3,4.3,45])
i = i+1

## 4 ##
syn_4_D = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer4D = AnnotationLayerConfig(name='annos4D', color=blues[i],
                                   mapping_rules=points)
sb4_D = StateBuilder(layers = [anno_layer4D], resolution=[4.3,4.3,45])
i = i+1

## 5 ##
syn_5_D = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer5D = AnnotationLayerConfig(name='annos5D', color=blues[i],
                                   mapping_rules=points)
sb5_D = StateBuilder(layers = [anno_layer5D], resolution=[4.3,4.3,45])
i = i+1

desc_states = [sb1_D,sb2_D,sb3_D,sb4_D,sb5_D]
desc_syns = [syn_1_D,syn_2_D,syn_3_D,syn_4_D,syn_5_D]
# chained_sb_L = ChainedStateBuilder([sb1,sb2,sb3,sb4,sb5])
# chained_sb.render_state([syn_1,syn_2,syn_3,syn_4,syn_5], return_as='html')

IndexError: single positional indexer is out-of-bounds

In [153]:
# create a neuroglancer state with different colored synapses for top x presynaptic partners
## SENSORY
df = synapse_df
df = df[df.pre_pt_root_id.isin(sensory_df.SegID.to_list())]
precounts = pd.DataFrame(df['pre_pt_root_id'].value_counts().rename_axis('SegID').reset_index(name='counts'))

#reds = ['#fdd4c2', '#fca082', '#fb694a', '#e32f27', '#b11218']
reds = ['#b11218', '#e32f27', '#fb694a', '#fca082', '#fdd4c2'] #dark to light
blues = ['#00035b', '#0504aa', '#0165fc', '#0485d1', '#a2cffe' ] #dark to light
greens = ['#06470c', '#658b38', '#15b01a', '#96f97b', '#c7fdb5']
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']
# df['color'] = reds

# img_source = client.info.image_source()
# img_layer = ImageLayerConfig(name='fanc_v4',
#                              source=img_source,
#                              )
# seg_source = client.info.segmentation_source()
# seg_layer = SegmentationLayerConfig(name = 'seg',
#                                     source = seg_source,
#                                     fixed_ids = query_list)#precounts.SegID[0:2])
# points = PointMapper(point_column='pre_pt_position', set_position =True) ######################## change this to toggle rendering of pre- or post- synaptic points

## 1 ##
i = 0
syn_1_S = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer1S = AnnotationLayerConfig(name='annos1S', color=greens[i],
                                   mapping_rules=points)
# sb1_S = StateBuilder(layers = [img_layer, seg_layer, anno_layer1S], resolution=[4.3,4.3,45])
sb1_S = StateBuilder(layers = [anno_layer1S], resolution=[4.3,4.3,45])

i = i +1

## 2 ##
syn_2_S = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer2S = AnnotationLayerConfig(name='annos2S', color=greens[i],
                                   mapping_rules=points)
sb2_S = StateBuilder(layers = [anno_layer2S], resolution=[4.3,4.3,45])
i = i +1

## 3 ##
syn_3_S = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer3S = AnnotationLayerConfig(name='annos3S', color=greens[i],
                                   mapping_rules=points)
sb3_S = StateBuilder(layers = [anno_layer3S], resolution=[4.3,4.3,45])
i = i+1

## 4 ##
syn_4_S = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer4S = AnnotationLayerConfig(name='annos4S', color=greens[i],
                                   mapping_rules=points)
sb4_S = StateBuilder(layers = [anno_layer4S], resolution=[4.3,4.3,45])
i = i+1

## 5 ##
syn_5_S = df[df.pre_pt_root_id.isin([precounts.SegID.iloc[i]])]
anno_layer5S = AnnotationLayerConfig(name='annos5S', color=greens[i],
                                   mapping_rules=points)
sb5_S = StateBuilder(layers = [anno_layer5S], resolution=[4.3,4.3,45])
i = i+1

desc_states = [sb1_S,sb2_S,sb3_S,sb4_S,sb5_S]
desc_syns = [syn_1_S,syn_2_S,syn_3_S,syn_4_S,syn_5_S]
# chained_sb_L = ChainedStateBuilder([sb1,sb2,sb3,sb4,sb5])
# chained_sb.render_state([syn_1,syn_2,syn_3,syn_4,syn_5], return_as='html')

In [156]:
chained_sb = ChainedStateBuilder([sb1_L,sb2_L,sb3_L,sb4_L,sb5_L,sb1_D,sb2_D,sb3_D,sb4_D,sb5_D,sb1_S,sb2_S,sb3_S,sb4_S,sb5_S])
chained_sb.render_state([syn_1_L,syn_2_L,syn_3_L,syn_4_L,syn_5_L,syn_1_D,syn_2_D,syn_3_D,syn_4_D,syn_5_D,syn_1_S,syn_2_S,syn_3_S,syn_4_S,syn_5_S], return_as='html')

In [18]:
# create a neuroglancer state with different colored synapses for top x presynaptic partners
## ALL divided by L/D/S
df = synapse_df
syn_L = df[df.pre_pt_root_id.isin(local_df.SegID.to_list())]
syn_D = df[df.pre_pt_root_id.isin(descending_df.SegID.to_list())]
syn_S = df[df.pre_pt_root_id.isin(sensory_df.SegID.to_list())]

blues = ['#00035b', '#0504aa', '#0165fc', '#0485d1', '#a2cffe' ] #dark to light
greens = ['#06470c', '#658b38', '#15b01a', '#96f97b', '#c7fdb5']
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']

img_source = client.info.image_source()
img_layer = ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()
seg_layer = SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = query_list)#precounts.SegID[0:2])
points = PointMapper(point_column='pre_pt_position', set_position =True) ######################## change this to toggle rendering of pre- or post- synaptic points

## LOCAL ##
anno_layer_L = AnnotationLayerConfig(name='annos_L', color=purples[3],
                                   mapping_rules=points)
sb_L = StateBuilder(layers = [img_layer, seg_layer, anno_layer_L], resolution=[4.3,4.3,45])

## DESCENDING ##
anno_layer_D = AnnotationLayerConfig(name='annos_D', color=blues[3],
                                   mapping_rules=points)
sb_D = StateBuilder(layers = [anno_layer_D], resolution=[4.3,4.3,45])

## SENSORY ##
anno_layer_S = AnnotationLayerConfig(name='annos_S', color=greens[3],
                                   mapping_rules=points)
sb_S = StateBuilder(layers = [anno_layer_S], resolution=[4.3,4.3,45])

chained_sb = ChainedStateBuilder([sb_L,sb_D,sb_S])
chained_sb.render_state([syn_L,syn_D,syn_S], return_as='html')
